# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
wpy_path="../Output/city.csv"
wpy_df=pd.read_csv(wpy_path)
wpy_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Sioux Lookout,75,CA,1613874961,58,50.1001,-91.9170,21.20,8.05
1,1,Tsiroanomandidy,78,MG,1613875236,94,-18.7667,46.0333,68.43,2.42
2,2,Giohar,100,SO,1613875236,92,2.7809,45.5005,68.58,8.23
3,3,Caravelas,100,BR,1613875236,84,-17.7125,-39.2481,78.37,7.27
4,4,Kapaa,40,US,1613875236,73,22.0752,-159.3190,77.00,14.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# gmaps.configure(api_key=g_key)
# fig = gmaps.figure()
# fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# fig.add_layer(heat_layer)

NameError: name 'heat_layer' is not defined

In [3]:
gmaps.configure(api_key=g_key)
humidity = wpy_df['Humidity'].astype(float)
locations = wpy_df[['Lat', 'Lng']]
locations
fig=gmaps.figure(center=(50.0, -35.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=10,
                                 point_radius=1)
heat_layer
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# reduced_cities_df = data_file_df.loc[(data_file_df["Wind Speed"] < 10) & (data_file_df["Cloudiness"] == 0) & \
#                                    (data_file_df["Max Temp"] > 40) & (data_file_df["Max Temp"] < 60)].dropna()

# reduced_cities_df


In [4]:
temp_con=wpy_df.loc[(wpy_df['Max Temp']>70) & (wpy_df['Max Temp']>80)]
# Wind speed less than 10 mph.
wind_con=temp_con.loc[(wpy_df['Wind Speed']<10)]
# Zero cloudiness.
cloud_con=wind_con.loc[(wpy_df['Cloudiness']==0)]
cloud_con

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
18,18,Geraldton,0,AU,1613875101,34,-28.7667,114.6000,84.2,5.75
161,161,Batemans Bay,0,AU,1613875276,85,-35.7167,150.1833,84.0,9.89
295,295,Tautira,0,PF,1613875307,74,-17.7333,-149.1500,80.6,6.91
372,372,Cabedelo,0,BR,1613875324,83,-6.9811,-34.8339,80.6,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = cloud_con.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df


,City,Country,Lat,Lng,Hotel Name
18,Geraldton,AU,-28.7667,114.6000,
161,Batemans Bay,AU,-35.7167,150.1833,
295,Tautira,PF,-17.7333,-149.1500,
372,Cabedelo,BR,-6.9811,-34.8339,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")


Retrieving Results for Index 18: Geraldton.
Closest hotel in Geraldton is Ocean Centre Hotel.
------------
Retrieving Results for Index 161: Batemans Bay.
Closest hotel in Batemans Bay is Bay Breeze Motel.
------------
Retrieving Results for Index 295: Tautira.
No hotel
------------
Retrieving Results for Index 372: Cabedelo.
Closest hotel in Cabedelo is Hotel Almagre.
------------


In [7]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
18,Geraldton,AU,-28.7667,114.6000,Ocean Centre Hotel
161,Batemans Bay,AU,-35.7167,150.1833,Bay Breeze Motel
295,Tautira,PF,-17.7333,-149.1500,
372,Cabedelo,BR,-6.9811,-34.8339,Hotel Almagre


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
hotel_info


['\n<dl>\n<dt>Name</dt><dd>Ocean Centre Hotel</dd>\n<dt>City</dt><dd>Geraldton</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bay Breeze Motel</dd>\n<dt>City</dt><dd>Batemans Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Tautira</dd>\n<dt>Country</dt><dd>PF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Almagre</dd>\n<dt>City</dt><dd>Cabedelo</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n']

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))